import necessary libraries

In [1]:
import numpy as np
import pickle


data_path = '../Data/training_data/period0/'

### The model we'll be using below

In [2]:
import numpy as np
import torch as t
import torch.nn as nn

from torch import LongTensor as LT
from torch import FloatTensor as FT

 
class Bundler(nn.Module):

    def forward(self, data):
        raise NotImplementedError

    def forward_i(self, data):
        raise NotImplementedError

    def forward_o(self, data):
        raise NotImplementedError


class Word2Vec(Bundler):

    def __init__(self, vocab_size=20000, embedding_size=300, padding_idx=0):
        # this instantiate the parent class
        super(Word2Vec, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        # define embeddings for the target word matrix and the context word matrix
        self.ivectors = nn.Embedding(self.vocab_size, self.embedding_size, padding_idx=padding_idx)
        self.ovectors = nn.Embedding(self.vocab_size, self.embedding_size, padding_idx=padding_idx)
        # define parameters with initial weights: 0 for the padding and uniformly sampled weights for the vocab
        self.ivectors.weight = nn.Parameter(t.cat([t.zeros(1, self.embedding_size), FT(self.vocab_size - 1, self.embedding_size).uniform_(-0.5 / self.embedding_size, 0.5 / self.embedding_size)]))
        self.ovectors.weight = nn.Parameter(t.cat([t.zeros(1, self.embedding_size), FT(self.vocab_size - 1, self.embedding_size).uniform_(-0.5 / self.embedding_size, 0.5 / self.embedding_size)]))
        # indicates that we do not exclude this as it is the main part of the graph - this should be treated as parameters
        self.ivectors.weight.requires_grad = True
        self.ovectors.weight.requires_grad = True

    def forward(self, data):
        return self.forward_i(data)

    def forward_i(self, data):
        v = LT(data)
        v = v.cuda() if self.ivectors.weight.is_cuda else v
        return self.ivectors(v)

    def forward_o(self, data):
        v = LT(data)
        v = v.cuda() if self.ovectors.weight.is_cuda else v
        return self.ovectors(v)


class SGNS(nn.Module):

    def __init__(self, embedding, vocab_size=20000, n_negs=20, weights=None):
        super(SGNS, self).__init__()
        self.embedding = embedding
        self.vocab_size = vocab_size
        self.n_negs = n_negs
        self.weights = None
        if weights is not None:
            wf = np.power(weights, 0.75)
            wf = wf / wf.sum()
            self.weights = FT(wf)

    def forward(self, iword, owords):
        batch_size = iword.size()[0]
        context_size = owords.size()[1]
        if self.weights is not None:
            nwords = t.multinomial(self.weights, batch_size * context_size * self.n_negs, replacement=True).view(batch_size, -1)
        else:
            nwords = FT(batch_size, context_size * self.n_negs).uniform_(0, self.vocab_size - 1).long()
        ivectors = self.embedding.forward_i(iword).unsqueeze(2)
        ovectors = self.embedding.forward_o(owords)
        nvectors = self.embedding.forward_o(nwords).neg()
##-------------- this is the line we need to modify: add the euclidean distance as a regularisation term
##-------------- we will also consider the consine similarity
        oloss = t.bmm(ovectors, ivectors).squeeze().sigmoid().log().mean(1)
        nloss = t.bmm(nvectors, ivectors).squeeze().sigmoid().log().view(-1, context_size, self.n_negs).sum(2).mean(1)
        return -(oloss + nloss).mean()

### code for training

In [3]:
import os
import pickle
import random
import argparse
import torch as t
import numpy as np

from tqdm import tqdm
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
#from model import Word2Vec, SGNS


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--name', type=str, default='sgns', help="model name")
    parser.add_argument('--data_dir', type=str, default='./data/', help="data directory path")
    parser.add_argument('--save_dir', type=str, default='./pts/', help="model directory path")
    parser.add_argument('--e_dim', type=int, default=300, help="embedding dimension")
    parser.add_argument('--n_negs', type=int, default=20, help="number of negative samples")
    parser.add_argument('--epoch', type=int, default=100, help="number of epochs")
    parser.add_argument('--mb', type=int, default=4096, help="mini-batch size")
    parser.add_argument('--ss_t', type=float, default=1e-5, help="subsample threshold")
    parser.add_argument('--conti', action='store_true', help="continue learning")
    parser.add_argument('--weights', action='store_true', help="use weights for negative sampling")
    parser.add_argument('--cuda', action='store_true', help="use CUDA")
    return parser.parse_args()


class PermutedSubsampledCorpus(Dataset):

    def __init__(self, datapath, ws=None):
        data = pickle.load(open(datapath, 'rb'))
        if ws is not None:
            self.data = []
            for iword, owords in data:
                if random.random() > ws[iword]:
                    self.data.append((iword, owords))
        else:
            self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        iword, owords = self.data[idx]
        return iword, np.array(owords)


def train(args):
    idx2word = pickle.load(open(os.path.join(args.data_dir, 'idx2word.dat'), 'rb'))
    wc = pickle.load(open(os.path.join(args.data_dir, 'wc.dat'), 'rb'))
    wf = np.array([wc[word] for word in idx2word])
    wf = wf / wf.sum()
    ws = 1 - np.sqrt(args.ss_t / wf)
    ws = np.clip(ws, 0, 1)
    vocab_size = len(idx2word)
    weights = wf if args.weights else None
    if not os.path.isdir(args.save_dir):
        os.mkdir(args.save_dir)
    model = Word2Vec(vocab_size=vocab_size, embedding_size=args.e_dim)
    modelpath = os.path.join(args.save_dir, '{}.pt'.format(args.name))
    sgns = SGNS(embedding=model, vocab_size=vocab_size, n_negs=args.n_negs, weights=weights)
    if os.path.isfile(modelpath) and args.conti:
        sgns.load_state_dict(t.load(modelpath))
    if args.cuda:
        sgns = sgns.cuda()
    optim = Adam(sgns.parameters())
    optimpath = os.path.join(args.save_dir, '{}.optim.pt'.format(args.name))
    if os.path.isfile(optimpath) and args.conti:
        optim.load_state_dict(t.load(optimpath))
    for epoch in range(1, args.epoch + 1):
        dataset = PermutedSubsampledCorpus(os.path.join(args.data_dir, 'train.dat'))
        dataloader = DataLoader(dataset, batch_size=args.mb, shuffle=True)
        total_batches = int(np.ceil(len(dataset) / args.mb))
        pbar = tqdm(dataloader)
        pbar.set_description("[Epoch {}]".format(epoch))
        for iword, owords in pbar:
            loss = sgns(iword, owords)
            optim.zero_grad()
            loss.backward()
            optim.step()
            pbar.set_postfix(loss=loss.item())
    idx2vec = model.ivectors.weight.data.cpu().numpy()
    pickle.dump(idx2vec, open(os.path.join(args.data_dir, 'idx2vec.dat'), 'wb'))
    t.save(sgns.state_dict(), os.path.join(args.save_dir, '{}.pt'.format(args.name)))
    t.save(optim.state_dict(), os.path.join(args.save_dir, '{}.optim.pt'.format(args.name)))


#if __name__ == '__main__':
 #   train(parse_args())

### We'll go through this line by line

In [78]:
import os
import pickle
import random

import argparse
import torch as t
import numpy as np

from tqdm import tqdm
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
#from model import Word2Vec, SGNS

In [79]:
# loading in idx2word and word counts and calculating the normalised frequency for words in the idx

with(open(os.path.join(data_path, 'idx2word.dat'), 'rb')) as handle:
    idx2word = pickle.load(handle)
with(open(os.path.join(data_path, 'wc.dat'), 'rb')) as handle:
    wc = pickle.load(handle)
wf = np.array([wc[word] for word in idx2word])
wf = wf / wf.sum()

In [80]:
# calculating the subsampling proba and obtaining vocab size

ss_t = 0.00001

ws = 1 - np.sqrt(ss_t / wf)
ws = np.clip(ws, 0, 1)
vocab_size = len(idx2word)

In [81]:
# do we wanna use weights for negsampling?
use_weights = True
weights = wf if use_weights else None

In [82]:
save_dir = data_path+'res/'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

In [83]:
# call and instantiate the model class
emb_dim = 300
model_name = 'period0'

model = Word2Vec(vocab_size=vocab_size, embedding_size=emb_dim)
modelpath = os.path.join(save_dir, '{}.pt'.format(model_name))

In [84]:
# call the SGNS model and instantiate
n_negs = 100

sgns = SGNS(embedding=model, vocab_size=vocab_size, n_negs=n_negs, weights=weights)

In [85]:
# do we wanna pick up from a previous learning?
# do we wanna use CUDA

conti = False
cuda = True

if os.path.isfile(modelpath) and conti:
    sgns.load_state_dict(t.load(modelpath))
if cuda:
    sgns = sgns.cuda()

In [86]:
# call the optimiser, we use Adam, can also use SGD

optim = Adam(sgns.parameters())
optimpath = os.path.join(save_dir, '{}.optim.pt'.format(model_name))
if os.path.isfile(optimpath) and conti:
    optim.load_state_dict(t.load(optimpath))

In [87]:
# train!
n_epoch = 10
mb = 1000

for epoch in range(1, n_epoch + 1):
    dataset = PermutedSubsampledCorpus(os.path.join(data_path, 'train.dat'), ws)
    dataloader = DataLoader(dataset, batch_size=mb, shuffle=True)
    total_batches = int(np.ceil(len(dataset) / mb))
    pbar = tqdm(dataloader)
    pbar.set_description("[Epoch {}]".format(epoch))
    for iword, owords in pbar:
        loss = sgns(iword, owords)
        optim.zero_grad()
        loss.backward()
        optim.step()
        pbar.set_postfix(loss=loss.item())

[Epoch 10]: 100%|██████████| 715/715 [04:12<00:00,  2.83it/s, loss=14.4]


In [118]:
model.ivectors.weight.data.cpu().numpy()

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08796434, -0.15097055, -0.03127842, ...,  0.05318276,
         0.10935717,  0.025937  ],
       [ 0.02457174, -0.08780659,  0.03407729, ..., -0.01212869,
         0.10207013,  0.02905024],
       ..., 
       [ 0.07203221, -0.05842225, -0.03681392, ...,  0.06110888,
         0.04586072,  0.04027474],
       [ 0.04385738, -0.05046888, -0.06967798, ...,  0.0561912 ,
         0.0412907 ,  0.03783019],
       [ 0.042177  , -0.0542722 , -0.04980376, ...,  0.06586516,
         0.05417005,  0.04906487]], dtype=float32)

In [131]:
vector3 = model.ivectors.weight.data.cpu().numpy()[3]
vector3

array([  7.99330696e-02,  -2.69200522e-02,  -2.09275130e-02,
        -8.55292976e-02,   2.46277452e-02,  -3.88276987e-02,
        -7.89520051e-03,  -3.93716507e-02,   1.25761896e-01,
        -4.69861589e-02,   5.03436439e-02,  -7.27194697e-02,
         4.30937447e-02,   3.55232432e-02,  -9.49791297e-02,
         1.61524601e-02,  -4.45144176e-02,  -2.65276376e-02,
         9.54658445e-03,  -5.86230867e-02,  -4.93730828e-02,
         2.66812686e-02,   7.24582896e-02,  -4.62642722e-02,
        -8.07512924e-02,   6.03754632e-02,   6.25171140e-02,
         9.46569070e-02,   1.60528179e-02,  -1.15590962e-02,
         3.73704843e-02,   7.11645484e-02,  -1.76927354e-02,
         4.17346247e-02,  -1.42934937e-02,  -7.50220343e-02,
         5.01422212e-02,   5.48556559e-02,   7.90656656e-02,
        -5.02137989e-02,  -6.85330182e-02,  -1.29830480e-01,
        -1.11729525e-01,  -5.05507067e-02,   4.18686457e-02,
         8.16334262e-02,   7.70266876e-02,  -4.39868458e-02,
         8.38929564e-02,

In [92]:
with open(data_path + 'word2idx.dat', 'rb') as handle:
    word2idx = pickle.load(handle)

In [113]:
word0, word1 = 'bad','good'
if word0 in idx2word and word1 in idx2word:
    comp_idx = word2idx[word0], word2idx[word1]
    print(comp_idx)
else:
    print('not found')

(728, 222)


In [114]:
array0, array1 = model.ivectors.weight.data.cpu().numpy()[comp_idx[0]],model.ivectors.weight.data.cpu().numpy()[comp_idx[1]]

In [115]:
from scipy import spatial

result = 1 - spatial.distance.cosine(array0, array1)
print(result)

0.955488026142


In [74]:
1 - spatial.distance.cosine([1,1,1,1,1,2], [1,1,1,1,1,0])

0.7453559924999299

In [96]:
res = [1 - spatial.distance.cosine(model.ivectors.weight.data.cpu().numpy()[1], model.ivectors.weight.data.cpu().numpy()[i]) for i in tqdm(range(1, vocab_size))]

100%|██████████| 19999/19999 [01:56<00:00, 171.53it/s]


In [98]:
np.mean(res), np.std(res)

(0.675554560940828, 0.097624454054056994)

Here we try to test adding regularisation

In [283]:
import numpy as np
import torch as t
import torch.nn as nn

from torch import LongTensor as LT
from torch import FloatTensor as FT

 
class Bundler(nn.Module):

    def forward(self, data):
        raise NotImplementedError

    def forward_i(self, data):
        raise NotImplementedError

    def forward_o(self, data):
        raise NotImplementedError


class Word2Vec(Bundler):

    def __init__(self, vocab_size=20000, embedding_size=300, padding_idx=0, vector_3 = None):
        # this instantiate the parent class
        super(Word2Vec, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        # define embeddings for the target word matrix and the context word matrix
        self.ivectors = nn.Embedding(self.vocab_size, self.embedding_size, padding_idx=padding_idx)
        self.ovectors = nn.Embedding(self.vocab_size, self.embedding_size, padding_idx=padding_idx)
        #self.vector_3 = nn.Embedding(self.vocab_size, 1, padding_idx=None)
        # define parameters with initial weights: 0 for the padding and uniformly sampled weights for the vocab
        self.ivectors.weight = nn.Parameter(t.cat([t.zeros(1, self.embedding_size), FT(self.vocab_size - 1, self.embedding_size).uniform_(-0.5 / self.embedding_size, 0.5 / self.embedding_size)]))
        self.ovectors.weight = nn.Parameter(t.cat([t.zeros(1, self.embedding_size), FT(self.vocab_size - 1, self.embedding_size).uniform_(-0.5 / self.embedding_size, 0.5 / self.embedding_size)]))
        #self.vector_3.weight = FT(vector_3)
        # indicates that we do not exclude this as it is the main part of the graph - this should be treated as parameters
        self.ivectors.weight.requires_grad = True
        self.ovectors.weight.requires_grad = True
        #self.vector_3.weight.requires_grad = False

    def forward(self, data):
        return self.forward_i(data)

    def forward_i(self, data):
        v = LT(data)
        v = v.cuda() if self.ivectors.weight.is_cuda else v
        return self.ivectors(v)

    def forward_o(self, data):
        v = LT(data)
        v = v.cuda() if self.ovectors.weight.is_cuda else v
        return self.ovectors(v)
    
    def forward_r(self, data):
        if data is not None:
            v = LT(data)
            v = v.cuda() if self.ivectors.weight.is_cuda else v
            return(self.ivectors(v))
        else:
            return(None)

class SGNS(nn.Module):

    def __init__(self, embedding, vocab_size=20000, n_negs=20, weights=None, test_tensor = None, vector3 = None):
        super(SGNS, self).__init__()
        self.embedding = embedding
        self.vocab_size = vocab_size
        self.n_negs = n_negs
        self.weights = None
        self.vector3 = FT(vector3).cuda()
        if weights is not None:
            wf = np.power(weights, 0.75)
            wf = wf / wf.sum()
            self.weights = FT(wf)

    def forward(self, iword, owords, rwords):
        batch_size = iword.size()[0]
        context_size = owords.size()[1]
        if self.weights is not None:
            nwords = t.multinomial(self.weights, batch_size * context_size * self.n_negs, replacement=True).view(batch_size, -1)
        else:
            nwords = FT(batch_size, context_size * self.n_negs).uniform_(0, self.vocab_size - 1).long()
        ivectors = self.embedding.forward_i(iword).unsqueeze(2)
        #print('ivectors', ivectors.shape)
        ovectors = self.embedding.forward_o(owords)
        #print('ovectors', ovectors.shape)
        nvectors = self.embedding.forward_o(nwords).neg()
        #print('nvectors', nvectors.shape)
        rvectors = self.embedding.forward_r(rwords)
##-------------- this is the line we need to modify: add the euclidean distance as a regularisation term
##-------------- we will also consider the consine similarity
        oloss = t.bmm(ovectors, ivectors).squeeze().sigmoid().log().mean(1)
        #print(oloss.shape)
        nloss = t.bmm(nvectors, ivectors).squeeze().sigmoid().log().view(-1, context_size, self.n_negs).sum(2).mean(1)
        #print(t.bmm(nvectors, ivectors).squeeze().sigmoid().log().view(-1, context_size, self.n_negs).sum(2).mean(1).shape)
        if rvectors is not None:
            rloss = 3*((rvectors.squeeze() - self.vector3)**2).sum()
            return -(oloss + nloss).mean() + rloss
        else:
            #print((-(oloss + nloss).mean()).shape)
            return -(oloss + nloss).mean()

In [284]:
emb_dim = 300
model_name = 'period0'

model_1 = Word2Vec(vocab_size=vocab_size, embedding_size=emb_dim)
modelpath = os.path.join(save_dir, '{}.pt'.format(model_name))

n_negs = 100

sgns_1 = SGNS(embedding=model_1, vocab_size=vocab_size, n_negs=n_negs, weights=weights, vector3=vector3)

In [285]:
sgns_1 = sgns_1.cuda()

In [286]:
optim_1 = Adam(sgns.parameters())
optimpath = os.path.join(save_dir, '{}.optim.pt'.format(model_name))
if os.path.isfile(optimpath) and conti:
    optim_1.load_state_dict(t.load(optimpath))

In [287]:
n_epoch = 10
mb = 1000

for epoch in range(1, n_epoch + 1):
    dataset = PermutedSubsampledCorpus(os.path.join(data_path, 'train.dat'), ws)
    dataloader = DataLoader(dataset, batch_size=mb, shuffle=True)
    total_batches = int(np.ceil(len(dataset) / mb))
    pbar = tqdm(dataloader)
    pbar.set_description("[Epoch {}]".format(epoch))
    for iword, owords in pbar:
        if 3 in iword:
            rwords = LT([3])
        else:
            rwords = None
        loss = sgns_1(iword, owords, rwords)
        optim_1.zero_grad()
        loss.backward()
        optim_1.step()
        pbar.set_postfix(loss=loss.item())


[Epoch 1]:   2%|▏         | 13/716 [00:04<04:18,  2.72it/s, loss=66.4]
Exception in thread Thread-39:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

[Epoch 1]:  16%|█▌        | 111/716 [00:40<03:38,  2.77it/s, loss=66.4]

KeyboardInterrupt: 

In [184]:
sum([0 == tens for tens in LT([0,1,1,2,5,6,3]).numpy()])

1

In [191]:
LT(vector3).shape

torch.Size([300])

In [210]:
print(LT(np.array([[2,5,8],[4,5,77]])).shape, LT([1,2,3]).shape)

torch.Size([2, 3]) torch.Size([3])


In [259]:
sgns_1(LT([1,2,3]), LT(np.array([[2,5,8],[4,5,77]]).T), None)

ivectors torch.Size([3, 300, 1])
ovectors torch.Size([3, 2, 300])
nvectors torch.Size([3, 200, 300])
torch.Size([3])
torch.Size([3])
torch.Size([])


tensor(70.0079, device='cuda:0', grad_fn=<NegBackward>)

In [196]:
n_epoch = 10
mb = 1000

for epoch in range(1, n_epoch + 1):
    dataset = PermutedSubsampledCorpus(os.path.join(data_path, 'train.dat'), ws)
    dataloader = DataLoader(dataset, batch_size=mb, shuffle=True)
    total_batches = int(np.ceil(len(dataset) / mb))
    pbar = tqdm(dataloader)
    pbar.set_description("[Epoch {}]".format(epoch))
    for iword, owords in pbar:
        print(iword.shape)
        print(owords.shape)
        print('----')


[Epoch 1]:   2%|▏         | 14/715 [00:00<00:23, 30.14it/s]

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1

[Epoch 1]:   6%|▋         | 46/715 [00:00<00:09, 68.31it/s]

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1

[Epoch 1]:  11%|█         | 79/715 [00:00<00:07, 89.40it/s]

...

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1

[Epoch 10]:  96%|█████████▌| 686/715 [00:04<00:00, 142.77it/s]

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1

[Epoch 10]: 100%|██████████| 715/715 [00:04<00:00, 143.07it/s]

torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([1000])
torch.Size([1000, 10])
----
torch.Size([323])
torch.Size([323, 10])
----
